# Reddit posts compared to Stock Price

In [ ]:
import pandas as pd

df= pd.read_csv("../input/reddit-finance-data/wallstreetbets/submissions_reddit.csv")

df.head(5)

## Count frecuent words

In [ ]:
#Download stopwords
import nltk
#nltk.download('stopwords')



In [ ]:
# remove stopwords 


def removeStopWords(texto):
    texto= str(texto)
    import nltk
    from nltk.corpus import stopwords
    stopwords= list(stopwords.words('english'))
    stopwords.append("im")
    lista2= texto.lower().split()
    for elemento in stopwords:
        if elemento in lista2:
            for i in range (lista2.count(elemento)):
                lista2.remove(elemento)  

    return lista2

#Remove signs and simbols

def RemoveNotAlphabetCharacters(lista):
    lista2=[]
    for palabra in lista:
        palabraN=""
        for letra in palabra:
            if letra.isalpha()==True:
                palabraN += letra
        if palabraN != "":
            lista2.append(palabraN)
    return lista2

#Create Dictionari with key: word , value: frec

def CountWords(lista):
    dic={}
    for palabra in lista:
        if palabra not in dic:
            dic[palabra] = 1
        elif palabra in dic:
            dic[palabra] += 1
    return dic

df["words"] = df["title"].apply(removeStopWords)

df["words"] = df["words"].apply(RemoveNotAlphabetCharacters)

df["words"] = df["words"].apply(CountWords)

In [ ]:
# I will create and auxiliary class called "Dic" , with an __add__ function
# The add function will allow us to sum dictionaries
# This way having a = { "apple" : 1 , "pineaple" : 3 } and b = { "apple" : 2 } (a and b are Dic objects);  
# a+b= { "apple" : 3 , "pineapple" : 3}

class Dic:
    def __init__ (self, dic):
        self.dic=(dic)
        self.cdic= dic.copy()
    
    
    def di(self):
        return (self.dic)
        
    def __add__ (self, other):
        
        if isinstance(other, Dic):
            other=(other).di()

            dic= self.dic
            
            cdic= dic.copy()

            for palabra in dict(other):
                if palabra not in cdic:
                    cdic[palabra] = other[palabra]
                elif palabra in dic:
                    cdic[palabra] += other[palabra]
        
        self.cdic=cdic
        return Dic((cdic))
    
    def __str__ (self):
        return str(self.dic)
    
def toAux(x):
    return Dic(x)
df["words"] = df["words"].apply(toAux)

In [ ]:
#I will group by day and have the dictionaries of df["words"] summed

#this function is to short "created" to only have days and not hours
def shortDate(t):
    t=str(t)
    return t[:10]

df["timestamp"]= df["created"].apply(shortDate)


df1 = df.groupby("timestamp").agg({"words" :"sum"})

def todict(Aux):
    return Aux.dic

df1["words"] = df1["words"].apply(todict)


In [ ]:
#EXECUTE PROGRAM TO SEE MOST COMMON WORDS IN A DAY

import operator


row_names = df1.index.values

date = input("Introduce the date where you want to search the most used words(YYYY-MM-DD): ")
x= dict(zip(list(row_names), range(len(list(row_names)))))[date]
df1_sorted = sorted(df1["words"][x].items(), key=operator.itemgetter(1), reverse=True)

for  y in range(0,10):
    print(df1_sorted[y])



## PLOT gme word appereance


In [ ]:
def countGME(dic):
    try: 
         return dic["gme"]
    except:
        return 0
    


In [ ]:
df1["gme"] = df1["words"].apply(countGME)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(50,25))
plt.xticks(range(200), size=5)



df1["gme"].plot(color="red")

plt.grid(True)





#plt.savefig("GME word appereance")

# yfinance , (superimpose two graphs)

In [ ]:

import pandas as pd





gme_data=pd.read_csv("../input/gmedata/gmedata.csv")


#df1=df1.reset_index()
gme_data= gme_data.reset_index()

gme_data["timestamp"]= (gme_data["Date"]).apply(shortDate)

gg= pd.merge(df1, gme_data, on='timestamp')

import matplotlib.pyplot as plt

plt.figure(figsize=(50,25))
plt.xticks(range(200), size=5)

gg['Close2']=gg['Close']*50

gg["gme"].plot(color="red")
gg["Close2"].plot(color="blue")

plt.grid(True)



## P-VALUE

#### The p-value roughly indicates the probability of an uncorrelated system producing datasets that have a Pearson correlation at least as extreme as the one computed from these datasets.


In [ ]:
import scipy.stats
print(f'The correlation coefficient is : {scipy.stats.pearsonr(gg["gme"], gg["Close2"])[0]}\n')
print('The p-value is:' , scipy.stats.pearsonr(gg["gme"], gg["Close2"])[1])
print(f'\nThe probability that the 2 graphs are not correlated is: {scipy.stats.pearsonr(gg["gme"], gg["Close2"])[1] *100} %')
print(f'The probability that the 2 graphs are correlated is: {100- scipy.stats.pearsonr(gg["gme"], gg["Close2"])[1] *100} %')


my_plot = gg.plot("gme", "Close2", kind="scatter")
plt.show() 

## Extra: 

get just messages containing gme chronologically

In [ ]:
def gmeinwords(dic):
    if "gme" in dic or "gamestop" in dic:
        return 1
    else:
        return 0

df["words"] = df["title"].apply(removeStopWords)

df["words"] = df["words"].apply(RemoveNotAlphabetCharacters)

df["gmeindic"] = df["words"].apply(gmeinwords)

In [ ]:
df3= df[df["gmeindic"] ==1]
df=df3.reset_index(drop=True)


In [ ]:
first=  pd.DataFrame()
first["timestamp"]= df["timestamp"] 
first["title"] = df["title"]
first["shortlink"] = df["shortlink"]

In [ ]:
for x in range(0,20):
    print(first["timestamp"][x] , first["title"][x], first["shortlink"][x])

...

...

##### By Pablo Llobregat (pallobre@gmail.com)